# Landcover Notebook

This notebook is being used to analyze the landcover in the area surrounding Lake Malawi. The work relies on data from the Sentinel-2 earth observation mission.

## Dependencies

Here we'll import dependencies that can be used in other cells in this notebook.

In [42]:
import json
import numpy as np
import pyproj
import rasterio
from rasterio.mask import mask
from shapely.geometry import box, shape
from shapely.ops import transform

## Helpers

Here we'll define some helper methods that can be used in other cells in this notebook.

In [ ]:
years = [2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]

# File helpers

def input_path(filename):
  return f"../data/input/{filename}"

def output_path(filename):
  return f"../data/output/{filename}"

# Coordinate helpers

def get_crs(path):
  with rasterio.open(path) as file:
    return file.crs

def peek_coordinates(coordinates):
  # Peek at the data
  first_three_coordinates = coordinates[:3]
  last_three_coordinates = coordinates[-3:]
  print(f"{first_three_coordinates}...{last_three_coordinates}")

# Raster helpers
def create_mask(path, type, coords):
  with rasterio.open(path) as file:
    geometry = {
      "type": type,
      "coordinates": coords
    }
    return mask(file, [geometry], crop=True, nodata=file.nodata)
  
def get_stats_from_mask(masked_data, file):
  valid_data = masked_data[masked_data != file.nodata]
  unique_values, counts = np.unique(valid_data, return_counts=True)
  print(f"Total valid pixels: {len(valid_data)}")
  print(f"Number of unique land cover classes: {len(unique_values)}")
  stats = {}
  for value, count in zip(unique_values, counts):
    percentage = (count / len(valid_data)) * 100
    stats[f"Class {int(value)}"] = {
      "pixels": count,
      "percentage": percentage 
    }
  return stats


## Lake Malawi Coordinates

We need to get the coordinates of the perimiter of the Lake Malawi. For this, we'll use a geojson file obtained from PyGeoAPI.

In [25]:
with open(input_path("lake_malawi.json"), 'r') as f:
    malawi_data = json.load(f)

malawi_coordinates = malawi_data['geometry']['coordinates'][0][0]
peek_coordinates(malawi_coordinates)

[35.2602047055542, -14.277474460510291]...[35.2602047055542, -14.277474460510291]


## Coordinate Reference System

In order to buffer the coordinates for Lake Malawi by 25km around the permiter and subsequently mask the geotiff files, we need to find out what coordinate reference system (CRS) we are working with. The CRS is embedded in the geotiff files.

In [26]:
crs = None
for year in years:
  crs_from_file = get_crs(input_path(f"malawi_{year}.tif"))
  if crs and crs != crs_from_file:
    raise "File has unexpected CRS"
  crs = crs_from_file
print(crs)

EPSG:32736


## Buffered Lake Malawi Coordinates

Using the Lake Malawi coordinates and the known CRS, we will define a 25km buffer, defining new coordinates for the region of interest, which will be the lake and the 25km surrounding it.

In [60]:
malawi_geometry = shape(malawi_data['geometry'])

# Define coordinate transformation
# WGS84 (lat/lon) to a projected CRS so we can work accurately with area
transformer_to_utm = pyproj.Transformer.from_crs("EPSG:4326", crs, always_xy=True)

# Transform to UTM (meters)
malawi_utm = transform(transformer_to_utm.transform, malawi_geometry)

# Buffer by 25km (25000 meters)
malawi_buffered = malawi_utm.buffer(25000)

# Get the coordinates of the buffered perimter
malawi_buffered_coordinates = [list(malawi_buffered.exterior.coords)]
peek_coordinates(malawi_buffered_coordinates)



[[(767343.5191822008, 8428923.241725704), (768019.0292322654, 8426759.23636284), (768495.732860627, 8424542.936137473), (768769.7102416275, 8422292.565182954), (768838.7085214938, 8420026.627788492), (768702.1603430429, 8417763.7562426), (768361.1885109307, 8415522.557624014), (765640.6495445707, 8401844.495205712), (765054.3271819914, 8399496.909871927), (764243.7995916855, 8397217.00289189), (763216.6597063933, 8395026.132181808), (761982.529658855, 8392944.821575956), (760552.9706427378, 8390992.568561694), (758941.3746086516, 8389187.66162914), (757162.8388098397, 8387547.008946638), (755234.0243727806, 8386085.979966893), (753173.0002176001, 8384818.261447652), (750999.0737904281, 8383755.729235668), (748732.6101933796, 8382908.337015079), (746394.8414065333, 8382284.0230624005), (744007.6673890926, 8381888.63588163), (741593.4509229918, 8381725.879416107), (739174.8081208308, 8381797.278350383), (736774.3965606353, 8382102.163827147), (734414.7030321748, 8382637.679713002), (7321

## Save Buffered Lake Malawi Coordinates

We will save a new geojson file with the buffered coordinates.

In [72]:
expanded_geojson = {
  "type": "FeatureCollection",
  "name": "lake_malawi_expanded_25km",
  "crs": {
    "type": "name",
    "properties": {
      "name": "urn:ogc:def:crs:OGC:1.3:CRS84"
    }
  },
  "features": [
    {
      "type": "Feature",
      "properties": {
        "name": "lake_malawi_expanded_25km",
        "buffer_distance": "25km"
      },
      "geometry": {
        "type": malawi_buffered.geom_type,
        "coordinates": malawi_buffered_coordinates
      }
    }
  ]
}

with open(output_path("lake_malawi_expanded_25km.json"), "w") as dst:
  json.dump(expanded_geojson, dst, indent=2)

## Analyze Land Cover within Buffered Lake Malawi Coordinates

The geotiff files contain the land cover data for a large region surrounding Lake Malawi. We'll mask the files to get the cover for the immediate area surrounding the lake.

In [71]:
# We can create the mask from any of the geotiff files.
masked_data, masked_transform = create_mask(
  input_path("malawi_2017.tif"),
  malawi_buffered.geom_type,
  malawi_buffered_coordinates
)
print(f"\nMasked data shape: {masked_data.shape}")
print(f"Masked transform: {masked_transform}")

for year in years:
  with rasterio.open(input_path(f"malawi_{year}.tif")) as file:
    bounding_box = box(*file.bounds)
    if bounding_box.intersects(malawi_buffered):
      stats = get_stats_from_mask(masked_data, file)
      print(year)
      for cls in stats:
        print(f"{cls}: {stats[cls]["pixels"]:,} pixels ({stats[cls]["percentage"]:.2f}%)")
    else:
      print("No overlap detected. Mask is not correct")

    output_profile = file.profile.copy()
    output_profile.update({
      'height': masked_data.shape[1],
      'width': masked_data.shape[2],
      'transform': masked_transform
    })
    with rasterio.open(output_path(f"malawi_{year}_masked.tif"), 'w', **output_profile) as dst:
      dst.write(masked_data)


Masked data shape: (1, 59350, 19442)
Masked transform: | 10.00, 0.00, 574420.00|
| 0.00,-10.00, 8975220.00|
| 0.00, 0.00, 1.00|
Total valid pixels: 594349727
Number of unique land cover classes: 8
2017
Class 1: 296,062,405 pixels (49.81%)
Class 2: 94,756,092 pixels (15.94%)
Class 4: 279,097 pixels (0.05%)
Class 5: 23,663,623 pixels (3.98%)
Class 7: 8,954,681 pixels (1.51%)
Class 8: 163,789 pixels (0.03%)
Class 10: 5,742 pixels (0.00%)
Class 11: 170,464,298 pixels (28.68%)
Total valid pixels: 594349727
Number of unique land cover classes: 8
2018
Class 1: 296,062,405 pixels (49.81%)
Class 2: 94,756,092 pixels (15.94%)
Class 4: 279,097 pixels (0.05%)
Class 5: 23,663,623 pixels (3.98%)
Class 7: 8,954,681 pixels (1.51%)
Class 8: 163,789 pixels (0.03%)
Class 10: 5,742 pixels (0.00%)
Class 11: 170,464,298 pixels (28.68%)
Total valid pixels: 594349727
Number of unique land cover classes: 8
2019
Class 1: 296,062,405 pixels (49.81%)
Class 2: 94,756,092 pixels (15.94%)
Class 4: 279,097 pixels (0